# Secured development

This demo shows:

- how to create a schema
- how to assing permissions to the schema
- how to work with ownership of schemas and objects

## How to create a schema and couple of objects to it
Let's recall that we have user *David* who will be a regular user. We will create second user with greater permissions as a develper of the database. Then we will observe how both users behave.

### Prep steps
Let's create second user called *Dev* now.

In [ ]:
use Demo
go

if exists(select * from sys.server_principals where name = 'Dev')
    drop login Dev
go

create login Dev with password = 'Pa$$w0rd', check_policy = off
go

create user Dev for login Dev
go

alter role db_owner add member Dev
go

### Let's create some schemas with different owners
- first statement shows recommended way of schema creation
- second schema show unreliable schema creation

In [10]:
use Demo
GO

execute as user = 'Dev'
go
create schema Sales authorization dbo
go
create schema Prod
go
revert
go

select 
    s.name as schema_name
    , d.name as owner_name
from sys.schemas as s 
    join sys.database_principals as d on s.principal_id = d.principal_id 
where s.name in ('dbo', 'Sales', 'Prod')

Commands completed successfully.

Commands completed successfully.

: Msg 2714, Level 16, State 6, Line 6
There is already an object named 'Sales' in the database.

: Msg 2759, Level 16, State 0, Line 6
CREATE SCHEMA failed due to previous errors.

: Msg 2714, Level 16, State 6, Line 8
There is already an object named 'Prod' in the database.

: Msg 2759, Level 16, State 0, Line 8
CREATE SCHEMA failed due to previous errors.

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.149

schema_name,owner_name
dbo,dbo
Prod,Dev
Sales,dbo


As we can see in previous result, the owner of the schema could be:
- dbo: when explicitly writen (*AUTHORIZATION* keyword) or when a member of **sysadmin** server role creates the schema
- creator: when the developer who created the schema is not a member of **sysadmin** role. It leads to a broken ownership chains

Let's create a couple of tables to both schemas. Then we'll assing some permissions to *David*.

### Creating objects, assigning permissions and observing the behavior

In [ ]:
use Demo
go

drop table if exists Sales.SalesCases
create table Sales.SalesCases
(
    Id int not null identity primary KEY
    , SomeValue int null    
)
GO

drop table if exists Prod.Product
create table Prod.Product
(
    Id int not null identity primary KEY
    , SomeValue int null    
)
GO


Now we'll assign permissions to allow David to use just objects in Sales schema. Immediately after it, David will execute two select statements.

In [9]:
grant select on schema::Sales to David
go

execute as user = 'David'
    select * from Sales.SalesCases
    select * from Prod.Product
revert

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.053

Id,SomeValue


: Msg 229, Level 14, State 5, Line 6
The SELECT permission was denied on the object 'Product', database 'Demo', schema 'Prod'.

As we can see, our user will retrieve an error when he tries to access data in different schema.

## Ownership chains
Now we'll create a view to the schema Sales owned by DBO. The view will union data from tables placed to two schemas. Then we will try to select data from teh view as David.

In [12]:
use Demo
GO

create or alter view Sales.viAllData
as
select Id, SomeValue from Sales.SalesCases
union all
select Id, SomeValue from Prod.Product
GO

execute as user = 'David'
select * from Sales.viAllData
revert

Commands completed successfully.

Commands completed successfully.

: Msg 229, Level 14, State 5, Line 12
The SELECT permission was denied on the object 'Product', database 'Demo', schema 'Prod'.

Total execution time: 00:00:00.080

Preceding select in David's context failed with an error. Read the error message carefully - it does not decline the access to whole view, but to the Product table located in Prod schema! It's not because David does not have permissions to read data from Prod schema. It's because the schema and objects in it have different owner! This what **broken ownership chains** means. Let's correct it.

### How to correct the ownership of schemas and objects
The correction of broken ownership is two-steps action.
1. We need to correct the ownership of objects
2. We need to correct the ownership of the schema itself

Following script does it:

In [ ]:
use Demo
GO
declare @schema nvarchar(30) = 'Prod'	-- parameter, schema for which the change has to be done
	, @objName nvarchar(100)
	, @sql nvarchar(1000) = 'ALTER AUTHORIZATION ON OBJECT::%s TO dbo'
	, @sqlinst nvarchar(1000)
declare crs cursor
for
select @schema + '.' + object_name(object_id) 
from sys.objects
where schema_id = SCHEMA_id(@schema)
	and type in ('FN', 'IF', 'U', 'V', 'P')

open crs
fetch crs into @objName
while @@FETCH_STATUS = 0
 begin
	set @sqlinst = replace(@sql, '%s', @objName)
	-- print(@sqlinst)
	exec(@sqlinst)
	fetch crs into @objName
 end
close crs
deallocate crs

set @sqlinst = replace(@sql, 'OBJECT::%s', 'SCHEMA::' + @schema)
exec(@sqlinst)
-- print(@sqlinst)

Let's test how David's work is changed:

In [14]:
use Demo
GO
execute as user = 'David'
select * from Sales.viAllData
revert

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.050

Id,SomeValue
